In [2]:
# pip install prophet

  Using cached prophet-1.1.4-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.0 MB)
  Using cached cmdstanpy-1.1.0-py3-none-any.whl (83 kB)
  Using cached LunarCalendar-0.0.9-py2.py3-none-any.whl (18 kB)
  Using cached convertdate-2.4.0-py3-none-any.whl (47 kB)
  Using cached holidays-0.27.1-py3-none-any.whl (598 kB)
  Using cached PyMeeus-0.5.12-py3-none-any.whl
  Using cached ephem-4.1.4-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import itertools

In [ ]:
from prophet import Prophet
# from pmdarima.arima import auto_arima
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

In [3]:
df = pd.read_csv('monthly_data.csv')

In [5]:
# df[["RemediMax",'VitaFlux',"SynaptraZol",'NeuroRevive','Immunoxen','RemediSol','Vitalisol','RestoraZen']] = df[["RemediMax",'VitaFlux',"SynaptraZol",'NeuroRevive','Immunoxen','RemediSol','Vitalisol','RestoraZen']].applymap(lambda x: None if x <= 1 else x)

In [6]:
df_validate = pd.read_csv('monthly_validate.csv')

In [7]:
df_ = pd.melt(df,id_vars='datum',value_vars=['RemediMax','VitaFlux','SynaptraZol','NeuroRevive','Immunoxen','RemediSol','Vitalisol','RestoraZen'])

In [8]:
df_.rename(columns={'datum':'ds','value':'y'},inplace=True)

In [9]:
df_ = df_.rename(columns={'datum':'ds','value':'y'})

In [22]:
#grid search on most important hyperparameter of prophet model https://facebook.github.io/prophet/docs/diagnostics.html
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

In [47]:
%%time

optimial_param = {}
count = 0


for mdl_ in df_['variable'].unique():
    
    
#     subset data set for each country & renaming columns as accepted by prophet
    df_train_ = df_[df_['variable']==mdl_].reset_index(drop=True)

    # df_train_.dropna(inplace=True)

    # df_train_.rename(columns={countr:'y'},inplace=True)

    # df_train_.reset_index(drop=True,inplace=True)

#     keep initial dynamic for each country
    # init_val = 2014 - df_train_['ds'].min().year - 5

    mape = [] 

    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(yearly_seasonality=True,weekly_seasonality=False, daily_seasonality=False,**params).fit(df_train_)  # Fit model with given params
        df_cv = cross_validation(m,horizon='12 days',period='12 days',initial="1430 days",parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1)
        mape.append(df_p['mape'].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results['mape'] = mape
    
#     saving best parameters in dictionary
    optimial_param[mdl_] = tuning_results.sort_values(by='mape').iloc[0].to_dict()
    
    print(count,' ',optimial_param[mdl_])
    count+=1
    
    

10:09:42 - cmdstanpy - INFO - Chain [1] start processing
10:09:43 - cmdstanpy - INFO - Chain [1] done processing
10:09:43 - cmdstanpy - INFO - Chain [1] start processing
10:09:43 - cmdstanpy - INFO - Chain [1] start processing
10:09:43 - cmdstanpy - INFO - Chain [1] done processing
10:09:43 - cmdstanpy - INFO - Chain [1] done processing
10:09:43 - cmdstanpy - INFO - Chain [1] start processing
10:09:43 - cmdstanpy - INFO - Chain [1] start processing
10:09:44 - cmdstanpy - INFO - Chain [1] done processing
10:09:44 - cmdstanpy - INFO - Chain [1] done processing
10:09:44 - cmdstanpy - INFO - Chain [1] start processing
10:09:44 - cmdstanpy - INFO - Chain [1] start processing
10:09:44 - cmdstanpy - INFO - Chain [1] done processing
10:09:44 - cmdstanpy - INFO - Chain [1] done processing
10:09:44 - cmdstanpy - INFO - Chain [1] start processing
10:09:44 - cmdstanpy - INFO - Chain [1] start processing
10:09:44 - cmdstanpy - INFO - Chain [1] done processing
10:09:44 - cmdstanpy - INFO - Chain [1]

0   {'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 0.01, 'mape': 0.07117076091216781}


10:10:59 - cmdstanpy - INFO - Chain [1] start processing
10:10:59 - cmdstanpy - INFO - Chain [1] start processing
10:10:59 - cmdstanpy - INFO - Chain [1] done processing
10:10:59 - cmdstanpy - INFO - Chain [1] start processing
10:11:00 - cmdstanpy - INFO - Chain [1] done processing
10:11:00 - cmdstanpy - INFO - Chain [1] start processing
10:11:04 - cmdstanpy - INFO - Chain [1] done processing
10:11:04 - cmdstanpy - INFO - Chain [1] start processing
10:11:04 - cmdstanpy - INFO - Chain [1] done processing
10:11:04 - cmdstanpy - INFO - Chain [1] start processing
10:11:05 - cmdstanpy - INFO - Chain [1] done processing
10:11:05 - cmdstanpy - INFO - Chain [1] start processing
10:11:07 - cmdstanpy - INFO - Chain [1] done processing
10:11:07 - cmdstanpy - INFO - Chain [1] start processing
10:11:07 - cmdstanpy - INFO - Chain [1] done processing
10:11:07 - cmdstanpy - INFO - Chain [1] start processing
10:11:08 - cmdstanpy - INFO - Chain [1] done processing
10:11:08 - cmdstanpy - INFO - Chain [1]

1   {'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.01, 'mape': 0.06319076674394897}


10:12:18 - cmdstanpy - INFO - Chain [1] start processing
10:12:18 - cmdstanpy - INFO - Chain [1] start processing
10:12:19 - cmdstanpy - INFO - Chain [1] done processing
10:12:19 - cmdstanpy - INFO - Chain [1] done processing
10:12:19 - cmdstanpy - INFO - Chain [1] start processing
10:12:19 - cmdstanpy - INFO - Chain [1] start processing
10:12:19 - cmdstanpy - INFO - Chain [1] done processing
10:12:19 - cmdstanpy - INFO - Chain [1] done processing
10:12:19 - cmdstanpy - INFO - Chain [1] start processing
10:12:19 - cmdstanpy - INFO - Chain [1] start processing
10:12:19 - cmdstanpy - INFO - Chain [1] done processing
10:12:19 - cmdstanpy - INFO - Chain [1] done processing
10:12:19 - cmdstanpy - INFO - Chain [1] start processing
10:12:19 - cmdstanpy - INFO - Chain [1] start processing
10:12:20 - cmdstanpy - INFO - Chain [1] done processing
10:12:20 - cmdstanpy - INFO - Chain [1] done processing
10:12:20 - cmdstanpy - INFO - Chain [1] start processing
10:12:20 - cmdstanpy - INFO - Chain [1]

...

5   {'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 0.01, 'mape': 0.26851144814221}


10:17:17 - cmdstanpy - INFO - Chain [1] start processing
10:17:17 - cmdstanpy - INFO - Chain [1] start processing
10:17:17 - cmdstanpy - INFO - Chain [1] done processing
10:17:17 - cmdstanpy - INFO - Chain [1] done processing
10:17:17 - cmdstanpy - INFO - Chain [1] start processing
10:17:17 - cmdstanpy - INFO - Chain [1] start processing
10:17:17 - cmdstanpy - INFO - Chain [1] done processing
10:17:17 - cmdstanpy - INFO - Chain [1] done processing
10:17:17 - cmdstanpy - INFO - Chain [1] start processing
10:17:17 - cmdstanpy - INFO - Chain [1] start processing
10:17:18 - cmdstanpy - INFO - Chain [1] done processing
10:17:18 - cmdstanpy - INFO - Chain [1] done processing
10:17:18 - cmdstanpy - INFO - Chain [1] start processing
10:17:18 - cmdstanpy - INFO - Chain [1] start processing
10:17:18 - cmdstanpy - INFO - Chain [1] done processing
10:17:18 - cmdstanpy - INFO - Chain [1] done processing
10:17:18 - cmdstanpy - INFO - Chain [1] start processing
10:17:18 - cmdstanpy - INFO - Chain [1]

6   {'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 0.1, 'mape': 0.2935171909455959}


10:18:40 - cmdstanpy - INFO - Chain [1] start processing
10:18:40 - cmdstanpy - INFO - Chain [1] start processing
10:18:40 - cmdstanpy - INFO - Chain [1] done processing
10:18:40 - cmdstanpy - INFO - Chain [1] done processing
10:18:40 - cmdstanpy - INFO - Chain [1] start processing
10:18:40 - cmdstanpy - INFO - Chain [1] start processing
10:18:40 - cmdstanpy - INFO - Chain [1] done processing
10:18:40 - cmdstanpy - INFO - Chain [1] done processing
10:18:40 - cmdstanpy - INFO - Chain [1] start processing
10:18:41 - cmdstanpy - INFO - Chain [1] start processing
10:18:41 - cmdstanpy - INFO - Chain [1] done processing
10:18:41 - cmdstanpy - INFO - Chain [1] start processing
10:18:41 - cmdstanpy - INFO - Chain [1] done processing
10:18:41 - cmdstanpy - INFO - Chain [1] start processing
10:18:42 - cmdstanpy - INFO - Chain [1] done processing
10:18:42 - cmdstanpy - INFO - Chain [1] start processing
10:18:42 - cmdstanpy - INFO - Chain [1] done processing
10:18:42 - cmdstanpy - INFO - Chain [1]

7   {'changepoint_prior_scale': 0.1, 'seasonality_prior_scale': 0.1, 'mape': 0.1599396004071527}
CPU times: user 25.5 s, sys: 4.54 s, total: 30 s
Wall time: 10min 15s


In [49]:
# optimal parameter for each country
pd.DataFrame(optimial_param)

,RemediMax,VitaFlux,SynaptraZol,NeuroRevive,Immunoxen,RemediSol,Vitalisol,RestoraZen
changepoint_prior_scale,0.500000,0.010000,0.100000,0.001000,0.500000,0.500000,0.500000,0.10000
seasonality_prior_scale,0.010000,0.010000,0.010000,0.100000,0.010000,0.010000,0.100000,0.10000
mape,0.071171,0.063191,0.103036,0.161452,0.155049,0.268511,0.293517,0.15994


In [10]:
optimial_param = {'RemediMax': {'changepoint_prior_scale': 0.5,
  'seasonality_prior_scale': 0.01,
  'mape': 0.07117076091216781},
 'VitaFlux': {'changepoint_prior_scale': 0.01,
  'seasonality_prior_scale': 0.01,
  'mape': 0.06319076674394897},
 'SynaptraZol': {'changepoint_prior_scale': 0.1,
  'seasonality_prior_scale': 0.01,
  'mape': 0.10303578956797543},
 'NeuroRevive': {'changepoint_prior_scale': 0.001,
  'seasonality_prior_scale': 0.1,
  'mape': 0.16145204635506014},
 'Immunoxen': {'changepoint_prior_scale': 0.5,
  'seasonality_prior_scale': 0.01,
  'mape': 0.15504860233480294},
 'RemediSol': {'changepoint_prior_scale': 0.5,
  'seasonality_prior_scale': 0.01,
  'mape': 0.26851144814221},
 'Vitalisol': {'changepoint_prior_scale': 0.5,
  'seasonality_prior_scale': 0.1,
  'mape': 0.2935171909455959},
 'RestoraZen': {'changepoint_prior_scale': 0.1,
  'seasonality_prior_scale': 0.1,
  'mape': 0.1599396004071527}}

In [11]:
df_

,ds,variable,y
0,2017-01-30,RemediMax,127.69
1,2017-02-27,RemediMax,133.32
2,2017-03-30,RemediMax,137.44
3,2017-04-29,RemediMax,113.10
4,2017-05-30,RemediMax,101.79
...,...,...,...
475,2021-08-30,RestoraZen,103.10
476,2021-09-29,RestoraZen,88.10
477,2021-10-30,RestoraZen,76.90
478,2021-11-29,RestoraZen,48.40


In [12]:
d_mdl = {} 

for mdl_ in df_['variable'].unique():
    
#     subset data set for each country & renaming columns as accepted by prophet 
    df_train_ = df_[df_['variable']==mdl_].reset_index(drop=True)

    
#     extracting optimal param for each country
    changepoint_prior_scale=optimial_param[mdl_]['changepoint_prior_scale']
    
    seasonality_prior_scale=optimial_param[mdl_]['seasonality_prior_scale']
        
#    training model
    m = Prophet(changepoint_prior_scale= changepoint_prior_scale,seasonality_prior_scale=seasonality_prior_scale,yearly_seasonality=True,weekly_seasonality=False, daily_seasonality=False).fit(df_train_)
    
    d_mdl[mdl_] = m

06:47:08 - cmdstanpy - INFO - Chain [1] start processing
06:47:09 - cmdstanpy - INFO - Chain [1] done processing
06:47:09 - cmdstanpy - INFO - Chain [1] start processing
06:47:09 - cmdstanpy - INFO - Chain [1] done processing
06:47:09 - cmdstanpy - INFO - Chain [1] start processing
06:47:09 - cmdstanpy - INFO - Chain [1] done processing
06:47:09 - cmdstanpy - INFO - Chain [1] start processing
06:47:09 - cmdstanpy - INFO - Chain [1] done processing
06:47:09 - cmdstanpy - INFO - Chain [1] start processing
06:47:10 - cmdstanpy - INFO - Chain [1] done processing
06:47:10 - cmdstanpy - INFO - Chain [1] start processing
06:47:11 - cmdstanpy - INFO - Chain [1] done processing
06:47:11 - cmdstanpy - INFO - Chain [1] start processing
06:47:11 - cmdstanpy - INFO - Chain [1] done processing
06:47:11 - cmdstanpy - INFO - Chain [1] start processing
06:47:11 - cmdstanpy - INFO - Chain [1] done processing


In [13]:
df_predict = df_validate.copy()

In [14]:
def mape(actual,forecast):
    return np.mean(abs(actual - forecast)/actual)

In [16]:
df_predict

,Unnamed: 0,datum,RemediMax,VitaFlux,SynaptraZol,NeuroRevive,Immunoxen,RemediSol,Vitalisol,RestoraZen
0,60,2022-01-30,179.70,222.351,99.70,1660.612,295.2,23.0,386.0,41.30
1,61,2022-02-27,133.73,142.155,110.20,1001.212,249.4,12.0,226.0,69.50
2,62,2022-03-30,154.52,113.118,83.35,941.050,301.4,19.0,257.0,169.50
3,63,2022-04-29,161.39,100.165,88.10,647.650,299.4,22.0,259.0,179.10
4,64,2022-05-30,168.04,97.258,104.10,703.562,265.8,26.0,322.0,135.40
5,65,2022-06-29,151.54,101.627,103.20,610.000,193.0,25.0,142.0,156.04
6,66,2022-07-30,181.00,103.541,92.80,649.800,250.6,20.0,115.0,105.20
7,67,2022-08-30,181.91,88.269,84.20,518.100,237.0,26.0,145.0,97.30
8,68,2022-09-29,161.07,111.437,93.50,984.480,227.8,16.0,161.0,109.10
9,69,2022-10-30,44.37,37.300,20.65,295.150,86.0,7.0,37.0,11.13


In [18]:
df_output = pd.DataFrame()

for mdl_ in df_['variable'].unique():
    
    df_predict_ = df_predict[[mdl_,'datum']]
    
    df_predict_.rename(columns={mdl_:'y','datum':'ds'},inplace=True)
    
    df_tmp_ = d_mdl[mdl_].predict(df_predict_)[['ds','yhat']]
    
    df_tmp_['model'] = mdl_
    
    df_tmp_['actual'] = df_predict_['y'].values
    
    df_output = pd.concat([df_output,df_tmp_])

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
df_output

,ds,yhat,model,actual
0,2022-01-30,137.656734,RemediMax,179.70
1,2022-02-27,145.667557,RemediMax,133.73
2,2022-03-30,148.835708,RemediMax,154.52
3,2022-04-29,147.889363,RemediMax,161.39
4,2022-05-30,145.314855,RemediMax,168.04
...,...,...,...,...
5,2022-06-29,143.533251,RestoraZen,156.04
6,2022-07-30,120.330267,RestoraZen,105.20
7,2022-08-30,97.735612,RestoraZen,97.30
8,2022-09-29,88.320486,RestoraZen,109.10


In [20]:
def mape(actual,forecast):
    return np.mean(abs(actual - forecast)/actual)

In [21]:
def smape(actual,forecast):
    return np.mean(2*abs(actual-forecast)/(abs(actual)+abs(forecast)))

In [22]:
d_error = {}

d_error['variable'] = []

d_error['mape'] = []

d_error['smape'] = []


for mdl_ in df_output['model'].unique():
    
    d_error['variable'].append(mdl_)
    
    d_error['mape'].append( mape(df_predict[mdl_],df_output[df_output['model']==mdl_]['yhat'].reset_index(drop=True) ))
    
    d_error['smape'].append( smape(df_predict[mdl_], df_output[df_output['model']==mdl_]['yhat'].reset_index(drop=True) ) )  

In [25]:
# pd.DataFrame(d_error)

In [33]:
pd.DataFrame(d_error).to_csv('prophet_error_no_imputation.csv')

In [37]:
df_output['ds'] = df_output['ds'].dt.date

In [35]:
df_output.rename(columns={'model':'variable','yhat':'forecast','ds':'date'},inplace=True)

In [27]:
df_output['model'] = 'prophet'

In [28]:
df_output['granularity'] = 'M'

In [29]:
df_output

,date,forecast,variable,actual,model,granularity
0,2022-01-30,137.656734,RemediMax,179.70,prophet,M
1,2022-02-27,145.667557,RemediMax,133.73,prophet,M
2,2022-03-30,148.835708,RemediMax,154.52,prophet,M
3,2022-04-29,147.889363,RemediMax,161.39,prophet,M
4,2022-05-30,145.314855,RemediMax,168.04,prophet,M
...,...,...,...,...,...,...
5,2022-06-29,143.533251,RestoraZen,156.04,prophet,M
6,2022-07-30,120.330267,RestoraZen,105.20,prophet,M
7,2022-08-30,97.735612,RestoraZen,97.30,prophet,M
8,2022-09-29,88.320486,RestoraZen,109.10,prophet,M


In [38]:
df_output.to_csv('prophet_result_final_no_imputation.csv',index=False)

In [132]:
mdl_fit.keys()

dict_keys(['RemediMax', 'VitaFlux', 'SynaptraZol', 'NeuroRevive', 'Immunoxen', 'RemediSol', 'Vitalisol', 'RestoraZen'])

In [19]:
mdl_fit[mdl_].predict()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2017-01-30,164.498773,65.815001,118.759467,164.498773,164.498773,-72.044383,-72.044383,-72.044383,-72.044383,-72.044383,-72.044383,0.0,0.0,0.0,92.454390
1,2017-01-30,164.498773,64.765714,119.911027,164.498773,164.498773,-72.044383,-72.044383,-72.044383,-72.044383,-72.044383,-72.044383,0.0,0.0,0.0,92.454390
2,2017-01-30,164.498773,66.462804,119.339478,164.498773,164.498773,-72.044383,-72.044383,-72.044383,-72.044383,-72.044383,-72.044383,0.0,0.0,0.0,92.454390
3,2017-01-30,164.498773,65.745526,118.717671,164.498773,164.498773,-72.044383,-72.044383,-72.044383,-72.044383,-72.044383,-72.044383,0.0,0.0,0.0,92.454390
4,2017-01-30,164.498773,65.708001,118.770394,164.498773,164.498773,-72.044383,-72.044383,-72.044383,-72.044383,-72.044383,-72.044383,0.0,0.0,0.0,92.454390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,2021-12-30,176.722622,124.665395,180.657712,176.722622,176.722622,-23.273545,-23.273545,-23.273545,-23.273545,-23.273545,-23.273545,0.0,0.0,0.0,153.449077
476,2021-12-30,176.722622,125.483874,181.420675,176.722622,176.722622,-23.273545,-23.273545,-23.273545,-23.273545,-23.273545,-23.273545,0.0,0.0,0.0,153.449077
477,2021-12-30,176.722622,126.934998,182.035847,176.722622,176.722622,-23.273545,-23.273545,-23.273545,-23.273545,-23.273545,-23.273545,0.0,0.0,0.0,153.449077
478,2021-12-30,176.722622,126.216956,179.882582,176.722622,176.722622,-23.273545,-23.273545,-23.273545,-23.273545,-23.273545,-23.273545,0.0,0.0,0.0,153.449077
